# Batch Processing Speech To Text - Sentiment Analysis

You need audio files in the audio files folder. Folder name - Audio Files in My Drive



In [ ]:
# Install the necessary libraries
!pip install git+https://github.com/openai/whisper.git
!pip install torch
!pip install soundfile
!pip install json


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-jrcp6pz6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-jrcp6pz6
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803557 sha256=39353f7a8adae73653f03083d16d0572233b9df37f0a78bb683419710cde9d18
  Stored in directory: /tmp/pip-ephem-wheel-cache-kld9xoje/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
ERROR: Could not find a version that satisfies the requirement json (from vers

In [ ]:
import os
import whisper
import json
import torch
import os
import json
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
import pandas as pd
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define paths to input and output folders in Google Drive
input_folder_path = "/content/drive/MyDrive/Audio Files"  # Change to your input folder path
output_folder_path = "/content/drive/MyDrive/Audio Files Transcribed"  # Change to your output folder path

# Load Whisper model
model = whisper.load_model("base")

# Ensure output folder exists
os.makedirs(output_folder_path, exist_ok=True)

# Loop through each file in the input folder
for filename in os.listdir(input_folder_path):
    if filename.endswith(".mp3"):  # Check for audio files only
        input_file_path = os.path.join(input_folder_path, filename)

        # Transcribe the audio file
        transcription = model.transcribe(input_file_path)

        # Extract only the 'text' part of the transcription
        transcription_text = {"text": transcription['text']}

        # Extract the numerical part of the filename using regular expressions
        match = re.search(r'\d+', filename)
        if match:
            output_filename = match.group(0) + ".json"  # Use only the numeric part and add .json extension
            output_file_path = os.path.join(output_folder_path, output_filename)

            # Save transcription text to JSON file
            with open(output_file_path, "w") as json_file:
                json.dump(transcription_text, json_file, indent=4)

            print(f"Transcription saved for {filename} as {output_filename}")
        else:
            print(f"No numeric identifier found in filename: {filename}")




# Load RoBERTa model pre-trained for sentiment analysis
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Function to analyze sentiment
def sentiment_analysis(text):
    # Encode the text into tokens
    encoded_text = tokenizer(text, return_tensors='pt')

    # Use the model to get the predictions
    with torch.no_grad():
        outputs = model(**encoded_text)

    # Get the prediction results
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    # Map predictions to sentiments
    sentiments = ['negative', 'neutral', 'positive']
    predicted_sentiment = sentiments[predictions.argmax().item()]
    confidence = predictions.max().item()

    return predicted_sentiment, confidence

# Path to the folder containing JSON files
folder_path = '/content/drive/MyDrive/Audio Files Transcribed'

# Initialize a dictionary to store the results
results = {}

# Iterate through each JSON file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        # Load the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)

            # Assuming 'transcript' is the key containing the text data in each JSON file
            text = data.get('text', '')
            if text:
                sentiment, confidence = sentiment_analysis(text)

                # Extract ID from filename (assuming filename like "1.json")
                file_id = os.path.splitext(filename)[0]

                # Add sentiment result and confidence to dictionary using file_id as key
                results[file_id] = {
                    'sentiment': sentiment,
                    'confidence': confidence
                }

# Define output file path
output_file_path = '/content/drive/MyDrive/sentiment_results.json'

# Save all results to a single JSON file
with open(output_file_path, 'w') as output_file:
    json.dump(results, output_file, indent=4)

print(f"Sentiment analysis results saved to {output_file_path}")

# Path to the JSON file with newly generated sentiments
new_sentiments_file_path = '/content/drive/MyDrive/sentiment_results.json'
# Path to the original JSON file with the 30 call transcripts and original sentiments
original_file_path = '/content/drive/MyDrive/transcripts_gpt.json'

# Load the original JSON file with sentiments
with open(original_file_path, 'r') as original_file:
    original_data = json.load(original_file)

# Load the new JSON file with sentiments
with open(new_sentiments_file_path, 'r') as new_file:
    new_data = json.load(new_file)

# Prepare to collect comparison results
comparison_results = []

# Iterate through each key in the new sentiments file
for key, new_sentiment_data in new_data.items():
    new_sentiment = new_sentiment_data.get('sentiment', '')
    new_confidence = new_sentiment_data.get('confidence', '')

    # Check if the key exists in the original data
    if key in original_data:
        original_sentiment = original_data[key]['sentiment']

        # Append comparison results to list
        comparison_results.append({
            'ID': key,
            'Original Sentiment': original_sentiment,
            'Code Generated Sentiment': new_sentiment,
            'Confidence': new_confidence
        })

# Convert comparison results to a DataFrame for tabular display
comparison_df = pd.DataFrame(comparison_results)

# Display the DataFrame in a tabular format
comparison_df


100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 41.2MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcription saved for transcript_23.mp3 as 23.json


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription saved for transcript_12.mp3 as 12.json


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


...

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Sentiment analysis results saved to /content/drive/MyDrive/sentiment_results.json


,ID,Original Sentiment,Code Generated Sentiment,Confidence
0,23,Neutral,neutral,0.777291
1,12,Negative,negative,0.912992
2,8,Neutral,neutral,0.820764
3,24,Negative,negative,0.962972
4,17,Neutral,neutral,0.796717
5,4,Positive,positive,0.990799
6,9,Negative,negative,0.858560
7,15,Negative,negative,0.974893
8,25,Positive,positive,0.988507
9,13,Positive,positive,0.990009


# Batch Processing Text To Speech - Speech to Text - Sentiment Analysis

In [5]:
!pip install gtts
!pip install pydub
# Install the necessary libraries
!pip install git+https://github.com/openai/whisper.git
!pip install torch
!pip install soundfile
!pip install json


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-jiizhhrx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-jiizhhrx
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [6]:
import os
import whisper
import json
import torch
from google.colab import drive
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import re
import pandas as pd
import json
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
drive.mount('/content/drive')
from gtts import gTTS


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:


# Path to the JSON file with transcripts
json_file_path = '/content/drive/MyDrive/Batch_Text To Speech_Speech To Text_Sentiment Analysis/content.json'

# Path to the directory where audio files will be saved
audio_files_directory = '/content/drive/MyDrive/Batch_Text To Speech_Speech To Text_Sentiment Analysis/Audio Files'

# Load JSON file with transcripts
with open(json_file_path, 'r') as file:
    transcripts = json.load(file)

# Ensure the directory for audio files exists
os.makedirs(audio_files_directory, exist_ok=True)

# Generate audio files for each transcript and save them in the specified folder
for key, data in transcripts.items():
    text = data["transcript"]
    tts = gTTS(text=text, lang='en')
    audio_file_path = os.path.join(audio_files_directory, f"transcript_{key}.mp3")
    tts.save(audio_file_path)

print(f"Audio files have been saved in: {audio_files_directory}")


# Define paths to input and output folders in Google Drive
input_folder_path = "/content/drive/MyDrive/Batch_Text To Speech_Speech To Text_Sentiment Analysis/Audio Files"  # Change to your input folder path
output_folder_path = "/content/drive/MyDrive/Batch_Text To Speech_Speech To Text_Sentiment Analysis/Transcription"  # Change to your output folder path

# Load Whisper model
model = whisper.load_model("base")

# Ensure output folder exists
os.makedirs(output_folder_path, exist_ok=True)

# Loop through each file in the input folder
for filename in os.listdir(input_folder_path):
    if filename.endswith(".mp3"):  # Check for audio files only
        input_file_path = os.path.join(input_folder_path, filename)

        # Transcribe the audio file
        transcription = model.transcribe(input_file_path)

        # Extract only the 'text' part of the transcription
        transcription_text = {"text": transcription['text']}

        # Extract the numerical part of the filename using regular expressions
        match = re.search(r'\d+', filename)
        if match:
            output_filename = match.group(0) + ".json"  # Use only the numeric part and add .json extension
            output_file_path = os.path.join(output_folder_path, output_filename)

            # Save transcription text to JSON file
            with open(output_file_path, "w") as json_file:
                json.dump(transcription_text, json_file, indent=4)

            print(f"Transcription saved for {filename} as {output_filename}")
        else:
            print(f"No numeric identifier found in filename: {filename}")




# Load RoBERTa model pre-trained for sentiment analysis
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Function to analyze sentiment
def sentiment_analysis(text):
    # Encode the text into tokens
    encoded_text = tokenizer(text, return_tensors='pt')

    # Use the model to get the predictions
    with torch.no_grad():
        outputs = model(**encoded_text)

    # Get the prediction results
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    # Map predictions to sentiments
    sentiments = ['negative', 'neutral', 'positive']
    predicted_sentiment = sentiments[predictions.argmax().item()]
    confidence = predictions.max().item()

    return predicted_sentiment, confidence

# Path to the folder containing JSON files
folder_path = '/content/drive/MyDrive/Batch_Text To Speech_Speech To Text_Sentiment Analysis/Transcription'

# Initialize a dictionary to store the results
results = {}

# Iterate through each JSON file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        # Load the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)

            # Assuming 'transcript' is the key containing the text data in each JSON file
            text = data.get('text', '')
            if text:
                sentiment, confidence = sentiment_analysis(text)

                # Extract ID from filename (assuming filename like "1.json")
                file_id = os.path.splitext(filename)[0]

                # Add sentiment result and confidence to dictionary using file_id as key
                results[file_id] = {
                    'sentiment': sentiment,
                    'confidence': confidence
                }

# Define output file path within 'sentiment' folder
output_folder_path = '/content/drive/MyDrive/Batch_Text To Speech_Speech To Text_Sentiment Analysis/Sentiment Analysis'
os.makedirs(output_folder_path, exist_ok=True)  # Create folder if it doesn't exist
output_file_path = os.path.join(output_folder_path, 'TTS_sentiment_analysis_results.json')

# Save all results to a single JSON file
with open(output_file_path, 'w') as output_file:
    json.dump(results, output_file, indent=4)

print(f"Sentiment analysis results saved to {output_file_path}")

with open(output_file_path, 'r') as output_file:
    saved_results = json.load(output_file)
    print(json.dumps(saved_results, indent=4))


Audio files have been saved in: /content/drive/MyDrive/Batch_Text To Speech_Speech To Text_Sentiment Analysis/Audio Files


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

Transcription saved for transcript_1.mp3 as 1.json


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription saved for transcript_2.mp3 as 2.json


...

Transcription saved for transcript_28.mp3 as 28.json


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription saved for transcript_29.mp3 as 29.json


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription saved for transcript_30.mp3 as 30.json
Sentiment analysis results saved to /content/drive/MyDrive/Batch_Text To Speech_Speech To Text_Sentiment Analysis/Sentiment Analysis/TTS_sentiment_analysis_results.json
{
    "1": {
        "sentiment": "positive",
        "confidence": 0.9896111488342285
    },
    "2": {
        "sentiment": "neutral",
        "confidence": 0.862250030040741
    },
    "3": {
        "sentiment": "negative",
        "confidence": 0.9656850099563599
    },
    "4": {
        "sentiment": "positive",
        "confidence": 0.9907993674278259
    },
    "5": {
        "sentiment": "neutral",
        "confidence": 0.8256492018699646
    },
    "6": {
        "sentiment": "negative",
        "confidence": 0.9630491733551025
    },
    "7": {
        "sentiment": "positive",
        "confidence": 0.9901731014251709
    },
    "8": {
        "sentiment": "neutral",
        "confidence": 0.8207636475563049
    },
    "9": {
        "sentiment": "negative",


# Speech to Text with Real tech Audio files

In [10]:
# Install the necessary libraries
!pip install git+https://github.com/openai/whisper.git
!pip install torch
!pip install soundfile
!pip install json

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0j9s3x7v
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0j9s3x7v
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [11]:
import os
import whisper
import json
import torch
from google.colab import drive
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import re
import pandas as pd
import os
import json
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Define paths to input and output folders in Google Drive
input_folder_path = "/content/drive/MyDrive/Test/Audio"  # Change to your input folder path
output_folder_path = "/content/drive/MyDrive/Test/Transcribe"  # Change to your output folder path

# Load Whisper model
model = whisper.load_model("base")

# Ensure output folder exists
os.makedirs(output_folder_path, exist_ok=True)

# Loop through each file in the input folder
for filename in os.listdir(input_folder_path):
    if filename.endswith(".mp3"):  # Check for audio files only
        input_file_path = os.path.join(input_folder_path, filename)

        # Transcribe the audio file
        transcription = model.transcribe(input_file_path)

        # Extract only the 'text' part of the transcription
        transcription_text = {"text": transcription['text']}

        # Extract the numerical part of the filename using regular expressions
        match = re.search(r'\d+', filename)
        if match:
            output_filename = match.group(0) + ".json"  # Use only the numeric part and add .json extension
            output_file_path = os.path.join(output_folder_path, output_filename)

            # Save transcription text to JSON file
            with open(output_file_path, "w") as json_file:
                json.dump(transcription_text, json_file, indent=4)

            print(f"Transcription saved for {filename} as {output_filename}")
        else:
            print(f"No numeric identifier found in filename: {filename}")




# Load RoBERTa model pre-trained for sentiment analysis
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Function to analyze sentiment
def sentiment_analysis(text):
    # Encode the text into tokens
    encoded_text = tokenizer(text, return_tensors='pt')

    # Use the model to get the predictions
    with torch.no_grad():
        outputs = model(**encoded_text)

    # Get the prediction results
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    # Map predictions to sentiments
    sentiments = ['negative', 'neutral', 'positive']
    predicted_sentiment = sentiments[predictions.argmax().item()]
    confidence = predictions.max().item()

    return predicted_sentiment, confidence

# Path to the folder containing JSON files
folder_path = '/content/drive/MyDrive/Test/Transcribe'

# Initialize a dictionary to store the results
results = {}

# Iterate through each JSON file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        # Load the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)

            # Assuming 'transcript' is the key containing the text data in each JSON file
            text = data.get('text', '')
            if text:
                sentiment, confidence = sentiment_analysis(text)

                # Extract ID from filename (assuming filename like "1.json")
                file_id = os.path.splitext(filename)[0]

                # Add sentiment result and confidence to dictionary using file_id as key
                results[file_id] = {
                    'sentiment': sentiment,
                    'confidence': confidence
                }

# Define output file path within 'sentiment' folder
output_folder_path = '/content/drive/MyDrive/Test/Sentiment'
os.makedirs(output_folder_path, exist_ok=True)  # Create folder if it doesn't exist
output_file_path = os.path.join(output_folder_path, 'sentiment_analysis_results.json')

# Save all results to a single JSON file
with open(output_file_path, 'w') as output_file:
    json.dump(results, output_file, indent=4)

print(f"Sentiment analysis results saved to {output_file_path}")

with open(output_file_path, 'r') as output_file:
    saved_results = json.load(output_file)
    print(json.dumps(saved_results, indent=4))

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

Transcription saved for transcript_8.mp3 as 8.json


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription saved for transcript_2.mp3 as 2.json


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription saved for transcript_12.mp3 as 12.json
Sentiment analysis results saved to /content/drive/MyDrive/Test/Sentiment/sentiment_analysis_results.json
{
    "8": {
        "sentiment": "neutral",
        "confidence": 0.8207636475563049
    },
    "2": {
        "sentiment": "neutral",
        "confidence": 0.862250030040741
    },
    "12": {
        "sentiment": "negative",
        "confidence": 0.9129921793937683
    }
}


# Text To Speech, Speech to Text with Real time Audio

In [13]:
!pip install gtts
!pip install pydub
!pip install git+https://github.com/openai/whisper.git
!pip install torch
!pip install soundfile
!pip install json

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-5p8fy4jo
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-5p8fy4jo
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [14]:
import os
import whisper
import json
import torch
from google.colab import drive
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import re
import pandas as pd
import os
import json
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import json
from gtts import gTTS
import os
import re
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Initialize a dictionary to store user input
user_inputs = {}

# Take user input and save to a dictionary
while True:
    key = input("Enter an identifier for the input (e.g., 1, 2, etc.): ")
    text = input("Enter the text to be converted to audio and analyzed: ")
    user_inputs[key] = {"transcript": text}

    # Ask if the user wants to add more inputs
    more = input("Do you want to add more inputs? (yes/no): ").strip().lower()
    if more != 'yes':
        break

# Save user inputs to a JSON file
json_file_path = '/content/drive/MyDrive/RealTech_TTS_STT_Analysis/content.json'
with open(json_file_path, 'w') as file:
    json.dump(user_inputs, file, indent=4)

print(f"User inputs have been saved to: {json_file_path}")

# Path to the directory where audio files will be saved
audio_files_directory = '/content/drive/MyDrive/RealTech_TTS_STT_Analysis/Real_TTS_Audio'
os.makedirs(audio_files_directory, exist_ok=True)

# Generate audio files for each transcript and save them in the specified folder
for key, data in user_inputs.items():
    text = data["transcript"]
    tts = gTTS(text=text, lang='en')
    audio_file_path = os.path.join(audio_files_directory, f"transcript_{key}.mp3")
    tts.save(audio_file_path)

print(f"Audio files have been saved in: {audio_files_directory}")

# Define paths to input and output folders in Google Drive
output_folder_path = "/content/drive/MyDrive/RealTech_TTS_STT_Analysis/Real_TTS_Transcribe"
os.makedirs(output_folder_path, exist_ok=True)

# Load Whisper model (if needed, you can uncomment this part if you have Whisper installed)
# import whisper
# model = whisper.load_model("base")

# Loop through each file in the input folder
for filename in os.listdir(audio_files_directory):
    if filename.endswith(".mp3"):  # Check for audio files only
        input_file_path = os.path.join(audio_files_directory, filename)

        # Transcribe the audio file
        # transcription = model.transcribe(input_file_path)

        # For now, simulate transcription for testing purposes
        transcription = {"text": user_inputs[filename.split('_')[1].split('.')[0]]["transcript"]}

        # Extract only the 'text' part of the transcription
        transcription_text = {"text": transcription['text']}

        # Extract the numerical part of the filename using regular expressions
        match = re.search(r'\d+', filename)
        if match:
            output_filename = match.group(0) + ".json"  # Use only the numeric part and add .json extension
            output_file_path = os.path.join(output_folder_path, output_filename)

            # Save transcription text to JSON file
            with open(output_file_path, "w") as json_file:
                json.dump(transcription_text, json_file, indent=4)

            print(f"Transcription saved for {filename} as {output_filename}")
        else:
            print(f"No numeric identifier found in filename: {filename}")

# Load RoBERTa model pre-trained for sentiment analysis
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name)

# Function to analyze sentiment
def sentiment_analysis(text):
    # Encode the text into tokens
    encoded_text = tokenizer(text, return_tensors='pt')

    # Use the model to get the predictions
    with torch.no_grad():
        outputs = model(**encoded_text)

    # Get the prediction results
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    # Map predictions to sentiments
    sentiments = ['negative', 'neutral', 'positive']
    predicted_sentiment = sentiments[predictions.argmax().item()]
    confidence = predictions.max().item()

    return predicted_sentiment, confidence

# Path to the folder containing JSON files
folder_path = '/content/drive/MyDrive/RealTech_TTS_STT_Analysis/Real_TTS_Transcribe'

# Initialize a dictionary to store the results
results = {}

# Iterate through each JSON file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        # Load the JSON file
        with open(file_path, 'r') as file:
            data = json.load(file)

            # Assuming 'transcript' is the key containing the text data in each JSON file
            text = data.get('text', '')
            if text:
                sentiment, confidence = sentiment_analysis(text)

                # Extract ID from filename (assuming filename like "1.json")
                file_id = os.path.splitext(filename)[0]

                # Add sentiment result and confidence to dictionary using file_id as key
                results[file_id] = {
                    'sentiment': sentiment,
                    'confidence': confidence
                }

# Define output file path within 'sentiment' folder
output_folder_path = '/content/drive/MyDrive/RealTech_TTS_STT_Analysis/Real_TTS_Sentiment'
os.makedirs(output_folder_path, exist_ok=True)  # Create folder if it doesn't exist
output_file_path = os.path.join(output_folder_path, 'TTS_sentiment_analysis_results.json')

# Save all results to a single JSON file
with open(output_file_path, 'w') as output_file:
    json.dump(results, output_file, indent=4)

print(f"Sentiment analysis results saved to {output_file_path}")

# Display the saved results
with open(output_file_path, 'r') as output_file:
    saved_results = json.load(output_file)
    print(json.dumps(saved_results, indent=4))


Enter an identifier for the input (e.g., 1, 2, etc.): 1
Enter the text to be converted to audio and analyzed: I was very surprised today when I came into the office to get my test results. I was asked to pay a co-pay and I didn't even get to see an actual doctor. This was very disappointing.
Do you want to add more inputs? (yes/no): no
User inputs have been saved to: /content/drive/MyDrive/RealTech_TTS_STT_Analysis/content.json
Audio files have been saved in: /content/drive/MyDrive/RealTech_TTS_STT_Analysis/Real_TTS_Audio
Transcription saved for transcript_1.mp3 as 1.json
Sentiment analysis results saved to /content/drive/MyDrive/RealTech_TTS_STT_Analysis/Real_TTS_Sentiment/TTS_sentiment_analysis_results.json
{
    "1": {
        "sentiment": "negative",
        "confidence": 0.9527900218963623
    }
}
